In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
with open('../static/model/sentiment_model.pickle', 'rb') as f:
    model=pickle.load(f)

In [3]:
with open('../static/model/categorization_model.pickle', 'rb') as f:
    modelc=pickle.load(f)

In [4]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [5]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw=file.read().splitlines()

In [6]:
vocab=pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens=vocab[0].tolist()

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [8]:
def preprocessing(text):
    data=pd.DataFrame([text],columns=['feedback'])
    data["feedback"] = data["feedback"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["feedback"] = data["feedback"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["feedback"] = data["feedback"].apply(remove_punctuation)
    data["feedback"] = data["feedback"].str.replace(r'\d+', '', regex=True)
    data["feedback"] = data["feedback"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["feedback"] = data["feedback"].apply(lambda x: " ".join([ps.stem(x) for x in x.split()]))
    return data["feedback"]

In [9]:
def vectorizer(ds,vocabulary):
    vectorized_list=[]

    for sentence in ds:
        sentence_list=np.zeros(len(vocabulary))
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i]=1
        vectorized_list.append(sentence_list)

    vectorized_list_new=np.asarray(vectorized_list,dtype=np.float32)
    return vectorized_list_new
    

In [33]:
def get_prediction(vectorized_txt):
    prediction = model.predict(vectorized_txt)
    if prediction == 1:
        return "negative"
    elif prediction == 10:
        return "neutral"
    else:
        return "positive"


In [34]:
def get_prediction_c(vectorized_txt):
    prediction = modelc.predict(vectorized_txt)
    if prediction == 1:
        return "service"
    elif prediction == 10:
        return "policy"
    else:
        return "claim"


In [35]:
txt="nothing response.the customer service is very dissatisfied"
preprocessed_txt=preprocessing(txt)
vectorized_txt=vectorizer(preprocessed_txt,tokens)
prediction=get_prediction(vectorized_txt)
prediction_category=get_prediction_c(vectorized_txt)
prediction


'negative'

In [36]:
prediction_category

'service'